In [1]:
import pandas as pd
import mysql.connector
maxNum = 0

In [2]:
df = pd.read_excel("112技優甄選.xls")

In [3]:
school = max(df['學校名稱'].unique(), key=len)
print(school)
temp = len(school)
if(temp > maxNum):
    maxNum = temp
print(maxNum)

嘉藥學校財團法人嘉南藥理大學
14


In [6]:
conn = mysql.connector.connect(user='project_tester', password='password', host='localhost', database='final_project')
cursor = conn.cursor()
cursor.execute("show tables;")
for row in cursor:
    print(row)

('friends',)
('members',)
('messages',)
('profiles',)


In [ ]:
mask = df['系名'].str.find("校區") > -1
df[mask][['校名', '系名']]